In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from scipy.stats import chi2_contingency
#This imported important packs 

In [2]:
df=pd.read_csv(r'/Users/kendraneumann/Python Labs/NSDUH2.0.csv')
df
#This uploaded the dataset

,QUESTID2,Heroin Age,Heroin Ever,Hep B/C Ever,Hep B/C Age,Unmet MH Need,K6 Psych Distress Score,Psych Distress,WHODAS,MDE Ever,Health,Gender,Race,Education,Employment,Insurance,Govt Program,Income Level,Below Poverty
0,39374400,NaN,No,NaN,NaN,No,8.0,No,8.0,No,1.0,Male,White,HS Grad,Full Time,No,Yes,2,Yes
1,23395500,NaN,No,NaN,NaN,No,9.0,No,0.0,Yes,1.0,Female,White,Some College,Part Time,Yes,No,2,No
2,41595500,NaN,No,NaN,NaN,No,3.0,No,2.0,No,2.0,Male,White,College Grad,Part Time,Yes,No,4,No
3,63105500,20.0,Yes,No,NaN,No,1.0,No,0.0,Yes,3.0,Male,White,Some College,Full Time,Yes,Yes,2,No
4,14815500,NaN,No,NaN,NaN,No,1.0,No,0.0,No,1.0,Female,Multiracial,NaN,NaN,Yes,No,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30762,63883400,NaN,No,NaN,NaN,No,15.0,Yes,20.0,Yes,4.0,Male,Hispanic,Some College,Full Time,Yes,No,2,No
30763,89614400,NaN,No,No,NaN,No,2.0,No,5.0,No,3.0,Male,White,HS Grad,Part Time,Yes,Yes,1,Yes
30764,43544400,NaN,No,NaN,NaN,No,9.0,No,8.0,No,1.0,Male,White,HS Grad,Full Time,Yes,No,4,No
30765,86864400,NaN,No,NaN,NaN,No,2.0,No,3.0,No,1.0,Female,Multiracial,Some College,Part Time,No,No,2,No


In [3]:
df['Heroin Ever'].value_counts()
#This counted the number of heroin users vs non-users

No     30267
Yes      500
Name: Heroin Ever, dtype: int64

In [4]:
def CI_prop(s, n):
    p = s / n
    se2 = p * (1-p) / n
    se = np.sqrt(se2)
    low = p - 1.96 * se
    up = p + 1.96 * se
    print('Sample proportion is:', p)
    print('95% CI is: ({}, {})'.format(low, up))
#This defined how to find CIs for proportions

In [5]:
CI_prop(500, 30767)
#This found the CI for the proportion of people in the US who use heroin

Sample proportion is: 0.016251178210420256
95% CI is: (0.0148383217734347, 0.01766403464740581)


In [6]:
df2=df[['Heroin Ever','Insurance']]
df2=df2.dropna()
df2
#This found necessary columns and dropped NaNs to compare insurance coverage based on heroin use

,Heroin Ever,Insurance
0,No,No
1,No,Yes
2,No,Yes
3,Yes,Yes
4,No,Yes
...,...,...
30762,No,Yes
30763,No,Yes
30764,No,Yes
30765,No,No


In [7]:
table2 = pd.crosstab(index=df2["Heroin Ever"], columns=df2["Insurance"]) 
table2
#This created a two-way table for insurance and heroin use

Insurance,No,Yes
Heroin Ever,,
No,2809,27458
Yes,85,415


In [9]:
table2['Total'] = table2['No'] + table2['Yes']
table2['Insured_per'] = table2['Yes'] / table2['Total']
table2['Uninsured_per'] = table2['No'] / table2['Total']
table2['Insured%']= table2['Insured_per']*100
table2['Uninsured%']= table2['Uninsured_per']*100
table2
#This found percentages to compare heroin use and insurance coverage

Insurance,No,Yes,Total,Insured_per,Uninsured_per,Insured%,Uninsured%
Heroin Ever,,,,,,,
No,2809,27458,30267,0.907193,0.092807,90.719265,9.280735
Yes,85,415,500,0.830000,0.170000,83.000000,17.000000


In [10]:
CI_prop(2809,30267)
#This found the confidence interval for the prop of uninsured non users

Sample proportion is: 0.09280734793669673
95% CI is: (0.08953836746515846, 0.096076328408235)


In [11]:
CI_prop(85,500)
#This found the confidence interval for the prop of uninsured heroin users

Sample proportion is: 0.17
95% CI is: (0.13707433341601116, 0.20292566658398886)


In [12]:
def CI_prop_diff(s1, s2, n1, n2):
    p1 = s1 / n1
    p2 = s2 / n2
    se2p1 = p1 * (1 - p1) / n1
    se2p2 = p2 * (1 - p2) / n2
    se2 = se2p1 + se2p2
    se = np.sqrt(se2)
    low = (p1 - p2) - 1.96 * se
    up = (p1 - p2) + 1.96 * se
    print("Proportion difference is:", p1 - p2)
    print("95% CI is: ({}, {})".format(low, up))
#This defined how to find CIs for proportion differences

In [13]:
CI_prop_diff(85,2809,500,30267)
#This found 95% CI in prop difference between uninsured heroin users and non users

Proportion difference is: 0.07719265206330328
95% CI is: (0.044105105262077615, 0.11028019886452894)


In [14]:
table2a=pd.crosstab(index=df2["Heroin Ever"], columns=df2["Insurance"]) 
#This recreated the table to find the p-value regarding heroin use and insurance

In [15]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(table2a)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#This found the p-value of the proportion difference in uninsured proportion based on heroin use

chi2: 33.49501134471069
p: 7.144701028671687e-09
dof: 1
expected: [[ 2846.96909026 27420.03090974]
 [   47.03090974   452.96909026]]


In [16]:
df3=df[['Heroin Ever', 'K6 Psych Distress Score']]
df3=df3.dropna()
df3
#This found necessary columns and dropped NaNs to compare heroin use and psychiatric distress score

,Heroin Ever,K6 Psych Distress Score
0,No,8.0
1,No,9.0
2,No,3.0
3,Yes,1.0
4,No,1.0
...,...,...
30762,No,15.0
30763,No,2.0
30764,No,9.0
30765,No,2.0


In [17]:
heroin=df3['Heroin Ever']=='Yes'
noheroin=df3['Heroin Ever']=='No'

heroin_df3=df3[heroin]
noheroin_df3=df3[noheroin]

heroin_k6=heroin_df3['K6 Psych Distress Score']
noheroin_k6=noheroin_df3['K6 Psych Distress Score']
#This created two dataframes for psych distress and heroin use

In [18]:
def CI_mean(data):
    se = np.std(data)/np.sqrt(len(data))
    low = np.mean(data)-1.96*se
    up = np.mean(data)+1.96*se
    print("Average is:", np.mean(data))
    print("95% CI of mean is: ({}, {})".format(low,up))
#This defined how to find the CI of means 

In [19]:
CI_mean(heroin_df3['K6 Psych Distress Score'])
#This found CI of mean of psych distress score among heroin users

Average is: 9.593939393939394
95% CI of mean is: (8.94168225543649, 10.246196532442298)


In [20]:
CI_mean(noheroin_df3['K6 Psych Distress Score'])
#This found CI of mean of psych distress score among non users

Average is: 5.580666306778746
95% CI of mean is: (5.508033953908055, 5.653298659649438)


In [21]:
def CI_mean_diff(list1, list2):
    s1 = np.var(list1)
    s2 = np.var(list2)
    n1 = len(list1)
    n2 = len(list2)
    se2 = s1/n1 + s2/n2
    se = np.sqrt(se2)

    diff = np.mean(list1) - np.mean(list2)
    low = diff - 1.96 * se
    up = diff + 1.96 * se
    print("The average difference is:", diff)
    print("The 95% CI is: ({}, {})".format(low, up))
#This defined how to find CI of mean differences

In [22]:
CI_mean_diff(heroin_df3['K6 Psych Distress Score'], noheroin_df3['K6 Psych Distress Score'])
#This found CI of mean diff in psych distress scores between users and non users

The average difference is: 4.0132730871606475
The 95% CI is: (3.3569844057013913, 4.669561768619904)


In [23]:
from scipy import stats

t_val, p_val = stats.ttest_ind(heroin_df3['K6 Psych Distress Score'], noheroin_df3['K6 Psych Distress Score'], equal_var=False)

print("Test statistic:", t_val)
print("p-value:", p_val)
#This found the p-value of the mean difference in psych distress scores between users and non users 

Test statistic: 11.973635009565363
p-value: 2.9020045294000394e-29


In [24]:
df4=df[['Heroin Ever','Psych Distress']]
df4=df4.dropna()
df4
#This found necessary columns and dropped NaNs to compare heroin use and 
#whether people are classified as have psychological distress (based on score)

,Heroin Ever,Psych Distress
0,No,No
1,No,No
2,No,No
3,Yes,No
4,No,No
...,...,...
30762,No,Yes
30763,No,No
30764,No,No
30765,No,No


In [25]:
table4 = pd.crosstab(index=df4["Heroin Ever"], columns=df4["Psych Distress"]) 

table4['Total'] = table4['No'] + table4['Yes']
table4['Dist_per'] = table4['Yes'] / table4['Total']
table4['NoDist_per'] = table4['No'] / table4['Total']
table4['Dist%']= table4['Dist_per']*100
table4['NoDist%']= table4['NoDist_per']*100

table4=table4.transpose()

table4
#This created a table, found comparable percentages, and transposed the table for visualization purposes.
#This compares psych distress based on heroin use

Heroin Ever,No,Yes
Psych Distress,,
No,22277.000000,324.000000
Yes,3657.000000,171.000000
Total,25934.000000,495.000000
Dist_per,0.141012,0.345455
NoDist_per,0.858988,0.654545
Dist%,14.101180,34.545455
NoDist%,85.898820,65.454545


In [26]:
CI_prop(171,495)
#This found 95% CI prop of psych distress positivity for heroin users

Sample proportion is: 0.34545454545454546
95% CI is: (0.30356372819110156, 0.38734536271798936)


In [27]:
CI_prop(3657,25934)
#This found 95% CI prop of psych distress positivity for non users

Sample proportion is: 0.14101179918254028
95% CI is: (0.1367759289761273, 0.14524766938895325)


In [28]:
CI_prop_diff(171,3657,495,25934)
#This found CI prop of difference between psych distress positivity in users vs non users

Proportion difference is: 0.20444274627200518
95% CI is: (0.16233831458833528, 0.24654717795567507)


In [29]:
table4a=pd.crosstab(index=df4["Heroin Ever"], columns=df4["Psych Distress"]) 
#This recreated the table to find the p-value regarding an association between psych distress and heroin use

In [30]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(table4a)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#The found the p value regarding an association between psych distress and heroin use

chi2: 162.26125207781874
p: 3.627546352099605e-37
dof: 1
expected: [[22177.69624276  3756.30375724]
 [  423.30375724    71.69624276]]


In [31]:
df5=df[['Heroin Ever','Unmet MH Need']]
df5=df5.dropna()
df5
#This found necessary columns and dropped NaNs to look at heroin use and unmet mental health need

,Heroin Ever,Unmet MH Need
0,No,No
1,No,No
2,No,No
3,Yes,No
4,No,No
...,...,...
30762,No,No
30763,No,No
30764,No,No
30765,No,No


In [32]:
table5 = pd.crosstab(index=df5["Heroin Ever"], columns=df5["Unmet MH Need"]) 

table5['Total'] = table5['No'] + table5['Yes']
table5['Need_per'] = table5['Yes'] / table5['Total']
table5['NoNeed_per'] = table5['No'] / table5['Total']
table5['Need%']= table5['Need_per']*100
table5['NoNeed%']= table5['NoNeed_per']*100
table5
#This created a table and found comparable percentages in heroin use and unmet MH need

Unmet MH Need,No,Yes,Total,Need_per,NoNeed_per,Need%,NoNeed%
Heroin Ever,,,,,,,
No,24170,1702,25872,0.065785,0.934215,6.578541,93.421459
Yes,416,77,493,0.156187,0.843813,15.618661,84.381339


In [33]:
CI_prop(77,493)
#This found CI prop of unmet mental health need for heroin users

Sample proportion is: 0.15618661257606492
95% CI is: (0.12414032539030899, 0.18823289976182084)


In [34]:
CI_prop(1702,25872)
#This found CI prop of unmet mental health need for non users

Sample proportion is: 0.06578540507111935
95% CI is: (0.06276455500412954, 0.06880625513810916)


In [35]:
CI_prop_diff(77,1702,493,25872)
#This found CI prop diff of unmet mental health need for users vs non users

Proportion difference is: 0.09040120750494557
95% CI is: (0.058212854680994214, 0.12258956032889692)


In [36]:
table5a = pd.crosstab(index=df5["Heroin Ever"], columns=df5["Unmet MH Need"]) 
#This recreated the table to find the p value regarding an association between heroin use and unmet need 

In [37]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(table5a)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#This found the p value regarding an association between heroin use and unmet need 

chi2: 61.4047357693541
p: 4.646914577453086e-15
dof: 1
expected: [[24126.26557937  1745.73442063]
 [  459.73442063    33.26557937]]


In [38]:
df6=df[['Heroin Age','Hep B/C Age']]
df6=df6.dropna()
df6
#This found necessary columns and dropped NaNs to compare heroin age and hep b/c age

,Heroin Age,Hep B/C Age
61,14.0,22.0
490,26.0,28.0
1100,21.0,20.0
1268,28.0,40.0
1756,18.0,33.0
...,...,...
29124,22.0,27.0
29130,15.0,45.0
29452,20.0,20.0
29639,12.0,17.0


In [39]:
from scipy import stats
corr = stats.pearsonr(df6['Heroin Age'], df6['Hep B/C Age'])
print('Correlation coefficient:', corr[0])
print('p-value:', corr[1])
#This found a correlation for the scatterplot between heroin age and hep b/c age

Correlation coefficient: 0.3139255960517344
p-value: 0.010262397351997883


In [40]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

x_vals = df6['Heroin Age'].values
y_vals = df6['Hep B/C Age']

reg_model = OLS(y_vals, sm.add_constant(x_vals)).fit()
display(reg_model.summary())
#This found a regression for the scatterplot between heroin age and hep b/c age

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Hep B/C Age   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     6.997
Date:                Tue, 22 Dec 2020   Prob (F-statistic):             0.0103
Time:                        20:33:25   Log-Likelihood:                -256.48
No. Observations:                  66   AIC:                             517.0
Df Residuals:                      64   BIC:                             521.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.4882      5.405      3.051      0.003       5.691      27.285
x1             0.6379      0.241      2.645      0.010       0.156       1.120
==============================================================================
Omnibus:                       12.707   Durbin-Watson:                   1.702
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               13.677
Skew:                           0.931   Prob(JB):                      0.00107
Kurtosis:                       4.228   Cond. No.                         82.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
df7=df[['Heroin Ever','Hep B/C Ever']]
df7=df7.dropna()
df7
#This found necessary columns and dropped NaNs to compare heroin use and hep b/c positivity

,Heroin Ever,Hep B/C Ever
3,Yes,No
11,No,No
12,No,No
13,No,No
24,No,No
...,...,...
30758,No,No
30759,No,No
30760,No,No
30761,No,No


In [42]:
table7 = pd.crosstab(index=df7["Heroin Ever"], columns=df7["Hep B/C Ever"]) 

table7['Total'] = table7['No'] + table7['Yes']
table7['HBC_per'] = table7['Yes'] / table7['Total']
table7['NoHBC_per'] = table7['No'] / table7['Total']
table7['HBC%']= table7['HBC_per']*100
table7['NoHBC%']= table7['NoHBC_per']*100

table7=table7.transpose()
table7
#This created a table, found comparable percentages, and transposed the table to compare heroin age and hep b/c age

Heroin Ever,No,Yes
Hep B/C Ever,,
No,8271.000000,131.000000
Yes,164.000000,70.000000
Total,8435.000000,201.000000
HBC_per,0.019443,0.348259
NoHBC_per,0.980557,0.651741
HBC%,1.944280,34.825871
NoHBC%,98.055720,65.174129


In [43]:
CI_prop(70,201)
#This found CI of prop of hep b/c pos for heroin users

Sample proportion is: 0.3482587064676617
95% CI is: (0.28239492627093576, 0.41412248666438767)


In [44]:
CI_prop(164,8435)
#This found CI of prop of hep b/c pos for non users

Sample proportion is: 0.01944279786603438
95% CI is: (0.01649614229198195, 0.02238945344008681)


In [45]:
CI_prop_diff(70,164,201,8435)
#This found CI of prop diff in hep b/c positivity between users and non users

Proportion difference is: 0.32881590860162735
95% CI is: (0.262886246681709, 0.3947455705215457)


In [46]:
table7a = pd.crosstab(index=df7["Heroin Ever"], columns=df7["Hep B/C Ever"]) 
#This recreted the table to find the p value regarding an association between heroin use and hep b/c positivity

In [47]:
from scipy import stats

chi2, p, dof, expected = stats.chi2_contingency(table7a)

print("chi2:", chi2)
print("p:", p)
print("dof:", dof)
print("expected:", expected)
#This found the p value regarding an association between heroin use and hep b/c positivity

chi2: 792.769717334136
p: 2.014019188089385e-174
dof: 1
expected: [[8.20644627e+03 2.28553729e+02]
 [1.95553729e+02 5.44627142e+00]]
